# DATA CLEANING #
This notebook shows the code and process of cleaning the dataset prior to EDA and preprocessing

## Step 1. Import Libraries ##

In [157]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
matplotlib.rcParams["figure.figsize"] = (20,10)
import re

## Step 2. Load and Inspect Dataset ##

In [ ]:
# Load the dataset and inspect the first 5 rows.
df1 = pd.read_csv("../data/mas_housing 2.csv")
df1.head()

In [ ]:
# Check the shape of the dataframe.
df1.shape

In [ ]:
# Check for missing values in each column.
df1.isnull().sum()

# Observation: Many nulls in dataset. Car Parks and Furnishing have high nulls.
# Imputation to be handled later.

## Step 3.  Clean *Location* Column

In [ ]:
# Determine counts for each location and look for duplicates due to variations in naming.

df1['Location'].value_counts().sort_index()

In [ ]:
# Remove redundant city names and normalize casing
# For location only retain area name, remove KL and commas. Standardized KLCC.

df1['Location'] = df1['Location'].str.replace(', Kuala Lumpur', '', regex=False)
df1['Location'] = df1['Location'].str.replace(r'\bklcc\b', 'KLCC', case=False, regex=True)
df1.head()


In [ ]:
# Some locations are named 'Other' which should be removed. Determine the count of 'Other' locations.

df1[df1['Location'] == 'Other'].value_counts()



In [ ]:
# Remove 'Other' locations from the dataset.

df1 = df1[df1['Location'] != 'Other']
df1.reset_index(drop=True, inplace=True)
df1['Location'].value_counts()

In [165]:
# Standardize capitalization for specific entries.

df1['Location'] = df1['Location'].replace('SANTUARI PARK PANTAI', 'Santuari Park Pantai')
df1['Location'] = df1['Location'].replace('cyberjaya', 'Cyberjaya')
df1['Location'] = df1['Location'].replace('U-THANT', 'U-Thant')
df1['Location'] = df1['Location'].replace('TAMAN MELATI', 'Taman Melati')
df1['Location'] = df1['Location'].replace('SEMARAK', 'Semarak')
df1['Location'] = df1['Location'].replace('Off Gasing Indah,', 'Gasing Indah')
df1['Location'] = df1['Location'].replace('kepong', 'Kepong')
df1['Location'] = df1['Location'].replace('ADIVA Desa ParkCity', 'Adiva Desa Park City')
df1['Location'] = df1['Location'].replace('Sungai Long SL8', 'Sungai Long')
df1['Location'] = df1['Location'].replace('taman connaught', 'Taman Connaught')
df1['Location'] = df1['Location'].replace('duta Nusantara', 'Duta Nusantara')
df1['Location'] = df1['Location'].replace('taman cheras perdana', 'Taman Cheras Perdana')


In [166]:
# Remove non-KL locations.

df1 = df1[~df1['Location'].str.contains('Landed Sd', na=False)]
df1.reset_index(drop=True, inplace=True)
df1 = df1[~df1['Location'].str.contains('Singapore', na=False)]
df1.reset_index(drop=True, inplace=True)

In [167]:
# Standardize location names for consistency.

df1['Location'] = df1['Location'].str.replace('Taman Yarl, UOG', 'Taman Yarl', regex=False)
df1['Location'] = df1['Location'].str.replace('Taman Yarl OUG', 'Taman Yarl', regex=False)
df1['Location'] = df1['Location'].str.replace('TamanYarl', 'Taman Yarl', regex=False)
df1['Location'] = df1['Location'].str.replace('Bandar Sri damansara', 'Bandar Sri Damansara', regex=False)

In [ ]:
# Check the final counts of locations after cleaning.

df1['Location'].value_counts().sort_index()

In [ ]:
# Set display options to show all rows
pd.set_option('display.max_rows', None)

# Show complete counts for each column
print("### Location Counts:")
print(df1['Location'].value_counts().sort_index())
print(f"\nTotal unique locations: {df1['Location'].nunique()}")


In [ ]:

# Remove records where Location is 'City Centre'
print(f"Records before removing City Centre: {len(df1)}")
df1 = df1[~df1['Location'].str.contains('City Centre', case=False, na=False)]
df1.reset_index(drop=True, inplace=True)
print(f"Records after removing City Centre: {len(df1)}")


## Step 4. Clean *Price* Column

In [ ]:
# Remove 'RM' and commas and convert to numeric type and show summary statistics

df1['Price'] = (
    df1['Price']
    .astype(str)
    .str.replace('RM', '', regex=False)
    .str.replace(',', '', regex=False)
    .str.strip()
    .replace({'': np.nan, '<NA>': np.nan})
    .astype(float)
    .astype('Int64')  # Nullable integer
)
 
print(df1['Price'].describe())

In [ ]:
# Remove records where Price is below 100,000 or above 10,000,000 and show summary statistics.

df1 = df1[(df1['Price'] >= 100000) & (df1['Price'] <= 10000000)]
df1.reset_index(drop=True, inplace=True)
print(df1['Price'].describe())


## Step 5. Clean *Rooms* Column

In [173]:
# Rooms column has mixed types, some are strings with '+' indicating multiple rooms. Sum the numbers in such cases, replace the output and convert to integer.

def sum_rooms(val):
    if isinstance(val, str) and '+' in val:
        parts = re.findall(r'\d+', val)
        if len(parts) == 2:
            return int(parts[0]) + int(parts[1])
    try:
        return int(val)
    except:
        return np.nan

df1.loc[:, 'Rooms'] = df1['Rooms'].apply(sum_rooms)
df1.loc[:, 'Rooms'] = df1['Rooms'].astype('Int64')

In [ ]:
# Remove missing value records and convert column to integer

# Check number of records before cleaning
print(f"Records before removing missing values: {len(df1)}")

# Remove missing values
df1 = df1.dropna(subset=['Rooms'])
df1.reset_index(drop=True, inplace=True)

# Convert to integer type
df1['Rooms'] = df1['Rooms'].astype(int)

# Check number of records after cleaning
print(f"Records after removing missing values: {len(df1)}")

# Verify the changes
print("\nRooms column data type:", df1['Rooms'].dtype)
print("\nUnique values in Rooms column:")
print(df1['Rooms'].value_counts().sort_index())

In [ ]:
# Remove records where Rooms is greater than 7 since they are outliers.

df1 = df1[df1['Rooms'] <= 7]
df1.reset_index(drop=True, inplace=True)
print(df1['Rooms'].describe())


## Step 6. Clean *Bathrooms* Column

In [ ]:
# Descriptive statistics and value counts for Bathrooms.

print(df1['Bathrooms'].describe())
print(df1['Bathrooms'].value_counts().sort_index())


In [ ]:
# Remove records where Bathrooms is greater than 7 since they are outliers.

df1 = df1[df1['Bathrooms'] <= 7]
df1.reset_index(drop=True, inplace=True)
print(df1['Bathrooms'].describe())


## Step 7. Clean *Car Parks* Column

In [ ]:
# Descriptive statistics and value counts for Car Parks

print(df1['Car Parks'].describe())
print(df1['Car Parks'].value_counts().sort_index())


In [ ]:
# Remove records where Car Parks is greater than 4 since they are outliers.

df1 = df1[df1['Car Parks'] <= 4]
df1.reset_index(drop=True, inplace=True)
print(df1['Car Parks'].describe())

## Step 8. Clean *Property Type* Column

In [ ]:
# List unique property types and their count.

print(df1['Property Type'].unique())
print(f"\nTotal unique property types: {df1['Property Type'].nunique()}")


In [ ]:
# Remove all records with 'Residential Land'.

df1 = df1[~df1['Property Type'].str.startswith('Residential Land', na=False)]
df1.reset_index(drop=True, inplace=True)
print(f"\nTotal unique property types: {df1['Property Type'].nunique()}")

In [ ]:
# Parse values into multiple columns of Type, Storeys, Position, Layout, Land Status

# Function to parse property strings
def parse_property(text):
    # Extract storeys if any (e.g., 2-sty, 3.5-sty)
    storey_match = re.search(r'(\d+(\.\d+)?)-sty', text)
    storeys = float(storey_match.group(1)) if storey_match else None

    # Extract bracket content (e.g., Corner, EndLot, Penthouse)
    bracket_match = re.search(r'\((.*?)\)', text)
    detail = bracket_match.group(1).strip() if bracket_match else None

    # Remove storey and bracketed details from base type
    base = re.sub(r'(\d+(\.\d+)?)-sty\s*', '', text)  # remove "X-sty"
    base = re.sub(r'\s*\(.*\)', '', base)            # remove "(...)"
    base = base.strip()

    # Identify categories
    positions = ['Corner', 'EndLot', 'Intermediate']
    layouts = ['Penthouse', 'Duplex', 'Triplex', 'Studio', 'SOHO']
    land_status = 'Land' if 'Land' in base else None
    position = detail if detail in positions else None
    layout = detail if detail in layouts else None

    return pd.Series([base, storeys, position, layout, land_status])

# Apply parsing to df1

df1[['MainType', 'Storeys', 'Position', 'Layout', 'LandStatus']] = df1['Property Type'].apply(parse_property)

# Standardise MainType (merge similar terms)

df1['MainType'] = df1['MainType'].replace({
    'Terrace/Link House': 'Terrace/Link House',
    'Cluster House': 'Cluster House',
    'Bungalow Land': 'Bungalow Land',
    'Bungalow': 'Bungalow',
    'Semi-detached House': 'Semi-detached House',
    'Condominium': 'Condominium',
    'Serviced Residence': 'Serviced Residence',
    'Apartment': 'Apartment',
    'Flat': 'Flat',
    'Townhouse': 'Townhouse'
})

# Preview
df1

## Step 9. Clean *Size* Column

In [ ]:
# Remove records with missing values in 'Size' and reset index

print(f"Records before removing missing Size: {len(df1)}")
df1 = df1.dropna(subset=['Size'])
df1.reset_index(drop=True, inplace=True)
print(f"Records after removing missing Size: {len(df1)}")

# Quick verification
print("Missing Size count:", df1['Size'].isna().sum())

In [ ]:
# Remove 'Built-up', 'Land area' (case-insensitive) and colons from Size column, trim whitespace

df1['Size'] = df1['Size'].astype(str)
df1['Size'] = df1['Size'].str.replace(r'(?i)\b(?:built-?up|land area)\b', '', regex=True)  # remove words
df1['Size'] = df1['Size'].str.replace(':', '', regex=False)                                    # remove colons
df1['Size'] = df1['Size'].str.replace(r'[\u00A0\s]+', ' ', regex=True).str.strip()             # normalize whitespace
df1.loc[df1['Size'].isin(['', 'nan', 'None']), 'Size'] = np.nan                                # empty -> NaN

# Quick check.

print("Sample cleaned Size values:")
print(df1['Size'].dropna().head(20))


In [ ]:
# List distinct Size values that contain letters (text-only entries)

s = df1['Size'].dropna().astype(str).str.strip()
text_mask = s.str.contains(r'[A-Za-z]', regex=True)
unique_text_sizes = pd.Series(s[text_mask].unique()).sort_values()

print(f"Distinct text-only Size values ({len(unique_text_sizes)}):")
for v in unique_text_sizes:
    print(v)


In [ ]:
# Remove apostrophes and bracketed content from Size column.

df1['Size'] = df1['Size'].astype(str)
df1['Size'] = df1['Size'].str.replace("'", "", regex=False)  # remove apostrophes
df1['Size'] = df1['Size'].str.replace(r'\([^)]*\)', '', regex=True)  # remove bracketed content
df1['Size'] = df1['Size'].str.strip()  # remove extra whitespace

# Quick check.

print("Sample cleaned Size values:")
print(df1['Size'].dropna().head(20))

In [ ]:
# Calculate multiplication for entries like '20 x 30' or '15X25' in Size column.

# Function to multiply numbers connected by x/X

def multiply_dimensions(val):
    if pd.isna(val):
        return np.nan
    # Convert to string and lowercase
    s = str(val).lower()
    # Look for pattern: number x number
    match = re.search(r'(\d+(?:\.\d+)?)\s*[xX]\s*(\d+(?:\.\d+)?)', s)
    if match:
        try:
            num1 = float(match.group(1))
            num2 = float(match.group(2))
            return num1 * num2
        except ValueError:
            return val
    return val

# Apply the multiplication.

df1['Size'] = df1['Size'].apply(multiply_dimensions)

# Quick check.

print("Sample multiplied Size values:")
print(df1['Size'].dropna().head(20))

In [ ]:
# Select entries with ranges like '800-900' and clean them.

# Function to handle ranges (keep first number before dash)

def clean_ranges(val):
    if pd.isna(val):
        return np.nan
    # Convert to string
    s = str(val)
    # Look for pattern: number-number
    match = re.search(r'(\d+(?:\.\d+)?)\s*-\s*\d+(?:\.\d+)?', s)
    if match:
        try:
            return float(match.group(1))  # Return only the first number
        except ValueError:
            return val
    return val

# Apply the range cleaning.

df1['Size'] = df1['Size'].apply(clean_ranges)

# Quick check.

print("Sample cleaned range values:")
print(df1['Size'].dropna().head(20))

In [ ]:
# Remove commas from Size column.

df1['Size'] = df1['Size'].astype(str).str.replace(',', '', regex=False)

# Quick check.

print("Sample values after removing commas:")
print(df1['Size'].dropna().head(20))

In [ ]:
# Remove 'sq. ft.' from Size column.

df1['Size'] = df1['Size'].astype(str).str.replace('sq. ft.', '', regex=False)
df1['Size'] = df1['Size'].str.strip()  # remove any trailing whitespace

# Quick check.

print("Sample values after removing 'sq. ft.':")
print(df1['Size'].dropna().head(20))

In [ ]:
# Remove all text from Size column.
 
df1['Size'] = df1['Size'].astype(str)
df1['Size'] = df1['Size'].str.replace(r'[A-Za-z]+', '', regex=True)  # remove all letters
df1['Size'] = df1['Size'].str.strip()  # remove trailing whitespace

# Convert to float and handle any remaining non-numeric values.

df1['Size'] = pd.to_numeric(df1['Size'], errors='coerce')

# Quick check.

print("Sample values after removing all text:")
print(df1['Size'].dropna().head(20))

In [ ]:
# Remove decimals by converting to integers.

df1['Size'] = df1['Size'].astype(float).round().astype('Int64')

# Quick check.

print("Sample values after removing decimals:")
print(df1['Size'].head(20))
print("\nColumn type:", df1['Size'].dtype)

In [ ]:
# Remove records with Size values outside reasonable range.

print(f"Records before filtering Size range: {len(df1)}")

df1 = df1[(df1['Size'] >= 400) & (df1['Size'] <= 10000)]
df1.reset_index(drop=True, inplace=True)

print(f"Records after filtering Size range: {len(df1)}")

# Verify the changes.

print("\nSize statistics after filtering:")
print(df1['Size'].describe())
print("\nSmallest sizes:", df1['Size'].nsmallest(5))
print("Largest sizes:", df1['Size'].nlargest(5))

In [ ]:
# Check for remaining values containing special symbols.

special_chars = r'[#xX*\+\-\\/\(\)\[\]\{\}\^\$\&\%\@\!\?\,\.]'
symbol_mask = df1['Size'].astype(str).str.contains(special_chars, regex=True)
special_values = pd.Series(df1.loc[symbol_mask, 'Size'].unique()).sort_values()

print(f"Values containing special symbols ({len(special_values)}):")
for v in special_values:
    print(v)

## Step 10. Clean *Furnishing* Column

In [ ]:
# Remove records with missing values in Furnishing column.

print(f"Records before removing missing Furnishing: {len(df1)}")
df1 = df1.dropna(subset=['Furnishing'])
df1.reset_index(drop=True, inplace=True)
print(f"Records after removing missing Furnishing: {len(df1)}")

# Quick verification.

print("\nUnique Furnishing values:")
print(df1['Furnishing'].value_counts())

In [ ]:
# Remove records with missing values and 'Unknown' in Furnishing column.

print(f"Records before cleaning Furnishing: {len(df1)}")

# Remove missing values and 'Unknown'.

df1 = df1.dropna(subset=['Furnishing'])
df1 = df1[~df1['Furnishing'].str.contains('Unknown', case=False, na=False)]
df1.reset_index(drop=True, inplace=True)

print(f"Records after cleaning Furnishing: {len(df1)}")

# Quick verification.

print("\nUnique Furnishing values:")
print(df1['Furnishing'].value_counts())

## Step 11. Rearrange columns of the data frame and remove *Layout* and *Land Status* columns since there are too many missing values

In [ ]:

# Ensure MainType exists (create if missing), then replace Property Type with MainType and rename to 'Type'.

if 'MainType' not in df1.columns:
    # parse_property is defined earlier in this notebook; reuse it to create MainType
    df1[['MainType', 'Storeys', 'Position', 'Layout', 'LandStatus']] = df1['Property Type'].apply(parse_property)

# Create new column 'Type' from MainType, drop original columns.

df1['Type'] = df1['MainType']
if 'Property Type' in df1.columns:
    df1.drop(columns=['Property Type'], inplace=True)
if 'MainType' in df1.columns:
    df1.drop(columns=['MainType'], inplace=True)

# Quick verification.

print("Columns now include 'Type':", 'Type' in df1.columns)
print("Sample Type values:", pd.Series(df1['Type'].unique()).sort_values()[:20])


In [ ]:
# List unique values of 'Type' column (sorted) and show counts.

print(f"Unique Type count: {df1['Type'].nunique()}\n")
print("Sorted unique values:")
for v in sorted(df1['Type'].dropna().unique()):
    print(v)

print("\nValue counts:")
print(df1['Type'].value_counts())

In [ ]:
# Remove 'Bungalow Land' and 'Cluster House' from Type (case-insensitive).

print(f"Records before removing Types: {len(df1)}")
mask_remove = df1['Type'].astype(str).str.contains(r'^(?:Bungalow Land|Cluster House)$', case=False, na=False)
removed = mask_remove.sum()
df1 = df1[~mask_remove].reset_index(drop=True)
print(f"Removed {removed} records. Records after removal: {len(df1)}")

# Quick verification.

print(f"\nUnique Type count after removal: {df1['Type'].nunique()}")
print(df1['Type'].value_counts())


In [ ]:
# Move 'Type' column to immediately after 'Location' (if both exist).

if 'Type' in df1.columns and 'Location' in df1.columns:
    cols = df1.columns.tolist()
    cols.remove('Type')
    lon_idx = cols.index('Location')
    cols.insert(lon_idx + 1, 'Type')
    df1 = df1[cols]
    print("Moved 'Type' after 'Location'.")
else:
    print("No change: 'Type' or 'Location' column not found.")
    
# Quick check.

df1.head()

In [ ]:
# Drop 'Layout' and 'LandStatus' if they exist, then move 'Price' to the end.

for col in ['Layout', 'LandStatus']:
    if col in df1.columns:
        df1.drop(columns=[col], inplace=True)
        print(f"Dropped column: {col}")

if 'Price' in df1.columns:
    cols = [c for c in df1.columns if c != 'Price'] + ['Price']
    df1 = df1[cols]
    print("Moved 'Price' to the end of the dataframe.")
else:
    print("Column 'Price' not found — no reordering performed.")

# Quick verification.

print(df1.columns.tolist())


## Step 12. Clean *Storeys* Column

In [ ]:
# List unique Type values with sum of missing Storeys beside each.

missing_storeys = (
    df1.groupby(df1['Type'].fillna('<MISSING TYPE>'))['Storeys']
       .apply(lambda s: s.isna().sum())
       .reset_index(name='Missing_Storeys')
       .sort_values('Missing_Storeys', ascending=False)
)
print(missing_storeys.to_string(index=False))


In [ ]:
# Robustly clean Storeys, fill missing with 1, then convert to numpy int64.

print(f"Missing Storeys before clean: {df1['Storeys'].isna().sum()}")

# Extract leading numeric (handles '3-sty', '3.5-sty', '3', etc.).

df1['Storeys'] = df1['Storeys'].astype(str).str.extract(r'(\d+(?:\.\d+)?)', expand=False)

# Convert to numeric (coerce non-numeric to NaN).

df1['Storeys'] = pd.to_numeric(df1['Storeys'], errors='coerce')

# Fill NaNs with 1, round any floats and convert to int64.

df1['Storeys'] = df1['Storeys'].fillna(1).round().astype('int64')

print(f"Missing Storeys after fill: {df1['Storeys'].isna().sum()}")
print(f"Storeys dtype: {df1['Storeys'].dtype}")


## Step 13. Clean *Position* Column

In [ ]:
# List unique Position values per Type and sum their occurrences.

tmp = df1.copy()
tmp['Position'] = tmp['Position'].fillna('<MISSING>')

# Counts per Type x Position.

pos_counts = (
    tmp.groupby(['Type', 'Position'])
       .size()
       .reset_index(name='Count')
       .sort_values(['Type', 'Count'], ascending=[True, False])
)
print("Counts by Type and Position:")
print(pos_counts.to_string(index=False))

# Summary per Type: unique positions (comma-separated), number of unique positions, and total occurrences.

summary = (
    pos_counts.groupby('Type')
              .agg(UniquePositions=('Position', lambda s: ', '.join(sorted(s.unique()))),
                   NumUnique=('Position', 'nunique'),
                   TotalCount=('Count', 'sum'))
              .reset_index()
              .sort_values('TotalCount', ascending=False)
)
print("\nSummary per Type:")
print(summary.to_string(index=False))


In [ ]:
# Fill missing Position values with 'Unknown' and normalize empty strings.

print(f"Position missing before: {df1['Position'].isna().sum()}")
df1['Position'] = df1['Position'].fillna('Unknown')
df1['Position'] = df1['Position'].replace(r'^\s*$', 'Unknown', regex=True)  # catch empty/blank strings
df1['Position'] = df1['Position'].astype(str)
print(f"Position missing after: {df1['Position'].isna().sum()}")
print(df1['Position'].value_counts())


In [ ]:
# List unique Position values and their counts.

positions = df1['Position'].fillna('Unknown').astype(str).str.strip()
print("Unique Position values (sorted):")
print(sorted(positions.unique()))
print("\nPosition value counts (sorted by name):")
print(positions.value_counts().sort_index())


## Step 14. Save cleaned data into csv file

In [ ]:

# Save cleaned dataframe to CSV
df1.to_csv("../data/cleaned_data_final.csv", index=False)
print("Saved cleaned_data to ../data/cleaned_data_final.csv")
